In [1]:
import healpy as hp
import numpy as np
import esutil
import os

def keepGoodRegion(des, hpInd = False, balrog=None):
    import healpy as hp
    import fitsio
    # 25 is the faintest object detected by DES
    # objects larger than 25 considered as Noise
    
    path = '/fs/scratch/PCON0008/warner785/bwarner/'
    #LSSGoldmask = fitsio.read(path+'Y1LSSmask_v2_il22_seeil4.0_nside4096ring_redlimcut.fits')
    #LSSGoldmask = fitsio.read(path+'Y1LSSmask_v1_il22seeil4.04096ring_redlimcut.fits')
    LSSGoldmask = fitsio.read(path+'MASK_Y3LSSBAOSOF_22_3_v2p2.fits')
    ringhp = hp.nest2ring(4096, [LSSGoldmask['PIXEL']])
    #Y1LSSmask_v1_il22seeil4.04096ring_redlimcut.fits
    #frac_cut = LSSGoldmask['FRAC'] > 0.8
    #ind_good_ring = LSSGoldmask['PIXEL'][frac_cut]
    ind_good_ring = ringhp
    
    # healpixify the catalog.
    nside=4096
    # Convert silly ra/dec to silly HP angular coordinates.
    phi = des['RA'] * np.pi / 180.0
    theta = ( 90.0 - des['DEC'] ) * np.pi/180.0

    hpInd = hp.ang2pix(nside,theta,phi,nest=False)
    keep = np.in1d(hpInd, ind_good_ring)
    des = des[keep]
    if hpInd is True:
        return ind_good_ring
    else:
        return des

import fitsio
cmass = esutil.io.read('/fs/scratch/PCON0003/warner785/bwarner/cmass-dr12v4-S-Reid-full.dat.fits')
train_sample = esutil.io.read('/fs/scratch/PCON0003/warner785/bwarner/cmass-dr12v4-S-Reid-full.dat.fits')
#train_sample = esutil.io.read('/global/cscratch1/sd/bwarner/galaxy_DR11v1_CMASS_South-photoObj.fits.gz')
print('total num of train', train_sample.size)
print('\n--------------------------------\n applying DES veto mask to CMASS\n--------------------------------')   
train_sample = keepGoodRegion(train_sample)

#fitsio.write( output_dir+'/cmass_in_st82.fits', train_sample)

print('num of train_sample after des veto', train_sample.size)

total num of train 258884

--------------------------------
 applying DES veto mask to CMASS
--------------------------------
num of train_sample after des veto 61031


In [6]:
import numpy 
# create binning in area option:
percentile  = [0, 10, 20, 30, 40, 50, 60, 70, 80, 90, 100]
# Calculate the bin edges
bin_edges = np.percentile(train_sample['MODELFLUX'], percentile)
pbin = bin_edges 
data = train_sample['MODELFLUX']
digitized = numpy.digitize(data, pbin)
pmean = [data[digitized == i].mean() for i in range(1, len(pbin))]

In [5]:
print(train_sample['MODELFLUX'])

[[ 1.2067251e-02  1.0100182e+00  5.1210017e+00  1.3349723e+01
   2.0785576e+01]
 [ 1.3008292e-01  2.3289137e+00  1.0380801e+01  2.4473183e+01
   3.2750881e+01]
 [ 5.1989114e-01  1.1966074e+00  5.3954234e+00  1.3482336e+01
   2.0196157e+01]
 ...
 [ 2.8186893e-01  1.0734087e+00  5.0721703e+00  1.3458070e+01
   1.8848003e+01]
 [-7.6981169e-01  1.1131186e+00  7.0957890e+00  1.8917547e+01
   3.0705122e+01]
 [ 1.3457608e-01  4.7537518e-01  3.8115921e+00  1.2212469e+01
   2.0130594e+01]]


In [ ]:
# flux to magnitude